# Model recovery attack: ablation studies case B

In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '..')
import mra
from mra import *

In [4]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

df = pd.read_excel('../../datasets/credit-card.xls', header=1, index_col=0).sample(frac=1)
min_values = df.drop(columns=["default payment next month"]).describe().transpose()['min'].to_numpy()
max_values = df.drop(columns=["default payment next month"]).describe().transpose()['max'].to_numpy()
x = df.drop(columns=["default payment next month"]).to_numpy()
x = (x-min_values)/(max_values-min_values)
y = df["default payment next month"].to_numpy().reshape((len(x), 1)).astype("float32")
train_ds = make_dataset(x, y, lambda t: t)

mra = mra(train_ds)

In [5]:
def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.BatchNormalization()(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    output = tf.keras.layers.Dense(64, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(64, activation="relu")(xin)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(xin, output)

In [6]:
log_train = mra.train(
    make_f,
    make_g,
    loss_fn=tf.keras.losses.binary_crossentropy,
    batch_size=32,
    epoch=5,
    lr=0.001,
    verbose=True, log_every=1000
    )

Iteration 0001: Training loss: 0.6953
Iteration 1001: Training loss: 0.3665
Iteration 2001: Training loss: 0.3447
Iteration 3001: Training loss: 0.3708
Iteration 4001: Training loss: 0.4857


In [7]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [8]:
log_attack_b = mra.attack(
    attack_iter=1,
    make_generator=make_generator,
    input_noise=False,
    input_z=True,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2353, RG-normal: 0.2181, reconstruction validation: 0.0409


In [9]:
log_attack_c = mra.attack(
    attack_iter=1,
    make_generator=make_generator,
    input_noise="normal",
    input_z=False,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2179, RG-normal: 0.2087, reconstruction validation: 0.0481


In [11]:
log_attack_d = mra.attack(
    attack_iter=1,
    make_generator=None,
    input_noise=None,
    input_z=None,
    model_leak=True,
    lr_x=0.001,
    lr_f=0.0001,
    epoch=200,
    iter_x=50,
    iter_f=1,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2276, RG-normal: 0.2242, reconstruction validation: 0.0296


In [12]:
log_attack_e = mra.attack(
    attack_iter=1,
    make_generator=make_generator,
    input_noise="normal",
    input_z=True,
    model_leak=False,
    lr_x=0.001,
    lr_f=0.001,
    epoch=200,
    iter_x=50,
    iter_f=50,
    verbose=True, log_every=1)

Iteration 0001: RG-uniform: 0.2177, RG-normal: 0.2203, reconstruction validation: 0.1360
